In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import sys
sys.path.append('../input/mmtest/mmtest')
import torch
import pandas as pd
from torch.utils.data import DataLoader

from utils import Config
from models import build_classifier
from dataset import LandmarkDataset
from tqdm import tqdm
import glob
import csv

def read_train_file(input_path):
    df = pd.read_csv(input_path)
    uniques = df['landmark_id'].unique()
    df['label'] = df['landmark_id'].map(dict(zip(uniques, range(len(uniques)))))
    return df, dict(zip(range(len(uniques)), uniques))


cfg_path = '../input/mmtest/mmtest/configs/regnet80.py'
checkpoint_path = '../input/google-landmark-test-model/epoch_100.pth'
img_path = '../input/landmark-recognition-2020/test'
_, mapping = read_train_file('../input/landmark-recognition-2020/train.csv')

cfg = Config.fromfile(cfg_path)
model = build_classifier(cfg.model)
model.load_state_dict(torch.load(checkpoint_path)['state_dict'])
model.eval()
model.cuda()
dataset = LandmarkDataset(img_path)
data_loader = DataLoader(dataset, batch_size=48, 
                          shuffle=False, num_workers=2)

results = []
with torch.no_grad():
    for img in tqdm(data_loader):
        img = img.cuda()
        res = model(img, return_loss=False)
        results.append(res)
        # del res,img
        # torch.cuda.empty_cache()

outputs = torch.cat(results)

softmax = torch.nn.Softmax(dim=1)
outputs = softmax(outputs)

confs, preds = outputs.max(1)

confs = confs.cpu().numpy()
preds = preds.cpu().numpy()

with open('submission.csv', 'w') as submission_csv:
    csv_writer = csv.DictWriter(submission_csv, fieldnames=['id', 'landmarks'])
    csv_writer.writeheader()
    for image_path, lable, score in zip(dataset.file_path, preds, confs):
        csv_writer.writerow({'id': image_path.split('/')[-1].split('.')[0], 'landmarks': f'{mapping[lable]} {score}'})